In [1]:
# =============================================================================
# INTEGER FACTORIZATION VIA SUBSET SUM REDUCTION
# Based on Hittmeir's Hyperbolic Sieve + Neural Network Optimization
# =============================================================================

!pip install -q torch numpy sympy

import numpy as np
import torch
import torch.nn as nn
from math import gcd, isqrt
from sympy import primerange, isprime, factorint
from typing import List, Tuple, Set, Dict
import heapq
import time

# =============================================================================
# PART 1: HYPERBOLIC SIEVE (From Hittmeir's Paper)
# =============================================================================

def modular_hyperbola(N: int, m: int) -> Set[Tuple[int, int]]:
    """
    Compute H_{N,m} = {(x,y) ∈ Z_m² : N ≡ xy mod m}
    """
    H = set()
    for x in range(1, m):
        if gcd(x, m) == 1:
            try:
                y = (N * pow(x, -1, m)) % m
                H.add((x, y))
            except:
                continue
    return H

def sieve_set(N: int, m: int, k: int = 1) -> Set[int]:
    """
    Compute L_{N,m,k} = {kx + y mod m : (x,y) ∈ H_{N,m}}
    This set contains (k*p + q) mod m for any factorization N = p*q
    """
    if gcd(N * k, m) != 1:
        return set()
    H = modular_hyperbola(N, m)
    L = set()
    for (x, y) in H:
        L.add((k * x + y) % m)
    return L

def build_sieve_modulus(target: int, max_primes: int = 50) -> Tuple[int, List[int]]:
    """
    Build modulus m as product of small primes until m > target
    """
    primes = []
    m = 1
    for p in primerange(3, 10000):
        if m > target or len(primes) >= max_primes:
            break
        primes.append(p)
        m *= p
    return m, primes

def compute_sieve_data(N: int, primes: List[int], k: int = 1):
    """
    Compute all sieve sets and CRT coefficients for the reduction
    """
    # L = ⌈2√(kN)⌉
    L_val = 2 * (isqrt(k * N) + 1)

    # Compute m = product of all primes
    m = 1
    for p in primes:
        m *= p

    # Compute shifted sieve sets and CRT coefficients
    shifted_sets = {}
    crt_coeffs = {}

    for r in primes:
        if gcd(N * k, r) != 1:
            continue

        # Sieve set for this prime
        L_r = sieve_set(N, r, k)
        # Shift by L
        shifted = sorted([(s - L_val) % r for s in L_r])
        shifted_sets[r] = shifted

        # CRT coefficient: M_r = (m/r) * ((m/r)^(-1) mod r)
        M_over_r = m // r
        gamma = pow(M_over_r, -1, r)
        M_r = (M_over_r * gamma) % m
        crt_coeffs[r] = M_r

    return shifted_sets, crt_coeffs, L_val, m

# =============================================================================
# PART 2: MCSS SOLVER USING NEURAL NETWORK / OPTIMIZATION
# =============================================================================

class MCSSNeuralSolver(nn.Module):
    """
    Neural network approach to solve Multiple-Choice Subset Sum
    Uses softmax to select one element from each class
    """

    def __init__(self, classes: List[torch.Tensor], M: int, target: float):
        super().__init__()
        self.classes = classes  # List of tensors, each containing class elements
        self.M = M
        self.target = target
        self.num_classes = len(classes)

        # Learnable logits for selecting from each class
        self.logits = nn.ParameterList([
            nn.Parameter(torch.zeros(len(c))) for c in classes
        ])

    def forward(self):
        """Compute the weighted sum using soft selection"""
        total = 0.0
        for logit, class_vals in zip(self.logits, self.classes):
            # Soft selection using softmax
            weights = torch.softmax(logit * 10, dim=0)  # Temperature scaling
            selected = (weights * class_vals).sum()
            total = total + selected
        return total % self.M

    def get_hard_selection(self):
        """Get the discrete selection (argmax)"""
        indices = []
        total = 0
        for logit, class_vals in zip(self.logits, self.classes):
            idx = torch.argmax(logit).item()
            indices.append(idx)
            total += class_vals[idx].item()
        return indices, int(total) % self.M

def solve_mcss_neural(classes: List[List[int]], M: int, target_range: Tuple[int, int],
                      max_iters: int = 1000, lr: float = 0.1) -> List[Tuple[int, List[int]]]:
    """
    Solve MCSS using gradient descent to minimize the sum
    """
    # Convert to tensors
    class_tensors = [torch.tensor(c, dtype=torch.float32) for c in classes]

    # Target is the middle of the range
    target = (target_range[0] + target_range[1]) / 2

    model = MCSSNeuralSolver(class_tensors, M, target)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    best_solutions = []
    seen = set()

    for iteration in range(max_iters):
        optimizer.zero_grad()

        # Forward pass
        soft_sum = model()

        # Loss: minimize the sum (we want smallest elements)
        loss = soft_sum

        # Backward pass
        loss.backward()
        optimizer.step()

        # Check discrete solution
        indices, hard_sum = model.get_hard_selection()

        if tuple(indices) not in seen:
            seen.add(tuple(indices))
            if target_range[0] <= hard_sum <= target_range[1]:
                best_solutions.append((hard_sum, indices))

        # Random restart occasionally
        if iteration % 100 == 99:
            with torch.no_grad():
                for logit in model.logits:
                    logit.add_(torch.randn_like(logit) * 0.5)

    return sorted(best_solutions, key=lambda x: x[0])

def solve_mcss_priority_queue(classes: List[List[int]], M: int,
                               target_range: Tuple[int, int],
                               max_candidates: int = 50000) -> List[Tuple[int, List[int]]]:
    """
    Solve MCSS using priority queue (branch and bound style)
    More reliable than neural approach for small instances
    """
    sorted_classes = [sorted(c) for c in classes]
    num_classes = len(classes)

    # Start with smallest elements
    initial_indices = tuple([0] * num_classes)
    initial_sum = sum(c[0] for c in sorted_classes)

    heap = [(initial_sum, initial_indices)]
    visited = {initial_indices}
    candidates = []

    min_target, max_target = target_range

    while heap and len(visited) < max_candidates:
        current_sum, indices = heapq.heappop(heap)

        # Check if in target range
        s = current_sum % M
        if s <= max_target:
            candidates.append((s, list(indices)))
        elif s >= M - max_target:
            actual = s - M
            if min_target <= actual <= max_target:
                candidates.append((actual, list(indices)))

        # Generate neighbors
        for i in range(num_classes):
            if indices[i] + 1 < len(sorted_classes[i]):
                new_indices = list(indices)
                new_indices[i] += 1
                new_indices = tuple(new_indices)

                if new_indices not in visited:
                    visited.add(new_indices)
                    delta = sorted_classes[i][new_indices[i]] - sorted_classes[i][indices[i]]
                    new_sum = current_sum + delta
                    heapq.heappush(heap, (new_sum, new_indices))

    return sorted(candidates, key=lambda x: abs(x[0]))

# =============================================================================
# PART 3: MAIN FACTORIZATION ALGORITHM
# =============================================================================

def factor_via_subset_sum(N: int, verbose: bool = True, use_neural: bool = False) -> Tuple[int, int]:
    """
    Factor N using the hyperbolic sieve reduction to subset sum

    Algorithm:
    1. Build hyperbolic sieve sets for small primes
    2. Reduce to Multiple-Choice Subset Sum problem
    3. Solve MCSS to find p + q
    4. Recover p and q from the sum
    """

    if verbose:
        print(f"\n{'='*60}")
        print(f"FACTORING N = {N}")
        print(f"{'='*60}")

    # Handle trivial cases
    if N % 2 == 0:
        return 2, N // 2

    sqrt_N = isqrt(N)
    if sqrt_N * sqrt_N == N:
        return sqrt_N, sqrt_N

    # Check small prime factors
    for p in primerange(3, min(1000, sqrt_N + 1)):
        if N % p == 0:
            return p, N // p

    # Estimate search bound: z = (p+q) - 2√N
    # For balanced primes, z ≈ (p-q)²/(4√N) which is small
    Lambda = min(sqrt_N, 10**8)  # Practical limit

    if verbose:
        print(f"√N ≈ {sqrt_N}")
        print(f"Search bound Λ = {Lambda}")

    # Build sieve modulus
    m, primes = build_sieve_modulus(Lambda, max_primes=20)

    # Check if gcd reveals factor
    g = gcd(N, m)
    if g > 1 and g < N:
        if verbose:
            print(f"Factor found via GCD!")
        return g, N // g

    if verbose:
        print(f"Using {len(primes)} primes: {primes}")
        print(f"Modulus m = {m}")

    # Compute sieve data
    k = 1  # Looking for p + q
    shifted_sets, crt_coeffs, L_val, m = compute_sieve_data(N, primes, k)

    if verbose:
        print(f"L = ⌈2√N⌉ = {L_val}")
        reduction = 1
        for r in primes:
            if r in shifted_sets:
                reduction *= len(shifted_sets[r]) / r
        print(f"Sieve reduction factor: {1/reduction:.2f}x")

    # Build MCSS classes
    classes = []
    class_primes = []
    for r in primes:
        if r not in shifted_sets or r not in crt_coeffs:
            continue
        M_r = crt_coeffs[r]
        class_elements = [(a * M_r) % m for a in shifted_sets[r]]
        classes.append(sorted(class_elements))
        class_primes.append(r)

    if verbose:
        print(f"\nMCSS Problem:")
        print(f"  Number of classes: {len(classes)}")
        total_combinations = 1
        for i, c in enumerate(classes):
            print(f"  Class {i} (prime {class_primes[i]}): {len(c)} elements")
            total_combinations *= len(c)
        print(f"  Total combinations: {total_combinations}")

    # Solve MCSS
    if verbose:
        print(f"\nSolving MCSS...")

    if use_neural:
        candidates = solve_mcss_neural(classes, m, (0, Lambda))
    else:
        candidates = solve_mcss_priority_queue(classes, m, (0, Lambda))

    if verbose:
        print(f"Found {len(candidates)} candidates to check")

    # Check candidates
    for z_val, indices in candidates:
        S = L_val + z_val  # p + q = L + z

        # Check if S² - 4N is a perfect square
        discriminant = S * S - 4 * N
        if discriminant >= 0:
            sqrt_disc = isqrt(discriminant)
            if sqrt_disc * sqrt_disc == discriminant:
                if (S + sqrt_disc) % 2 == 0:
                    p = (S + sqrt_disc) // 2
                    q = (S - sqrt_disc) // 2
                    if p * q == N and p > 1 and q > 1:
                        if verbose:
                            print(f"\n✓ FOUND FACTORIZATION!")
                            print(f"  z = {z_val}")
                            print(f"  p + q = {S}")
                            print(f"  p - q = {sqrt_disc}")
                        return min(p, q), max(p, q)

    if verbose:
        print(f"\n✗ No factor found in search range")
    return None, None

# =============================================================================
# PART 4: INTERACTIVE DEMO
# =============================================================================

def run_demo():
    """Interactive demonstration"""

    print("""
╔══════════════════════════════════════════════════════════════╗
║  INTEGER FACTORIZATION VIA SUBSET SUM REDUCTION              ║
║  ─────────────────────────────────────────────────────────── ║
║  Method: Hittmeir's Hyperbolic Sieve + MCSS Optimization     ║
╚══════════════════════════════════════════════════════════════╝
    """)

    # Test cases with balanced primes (close to each other)
    test_cases = [
        (101, 103),           # Tiny
        (1009, 1013),         # Small
        (10007, 10009),       # Medium
        (100003, 100019),     # Larger
        (1000003, 1000033),   # Even larger
    ]

    print("Running test cases with balanced primes (where this method excels):\n")

    results = []
    for p, q in test_cases:
        N = p * q
        print(f"\n{'─'*60}")

        start = time.time()
        found_p, found_q = factor_via_subset_sum(N, verbose=True)
        elapsed = time.time() - start

        success = found_p is not None and found_p * found_q == N
        results.append((N, success, elapsed))

        if success:
            print(f"\n✓ {N} = {found_p} × {found_q}")
        else:
            print(f"\n✗ Failed to factor {N}")
        print(f"Time: {elapsed:.4f}s")

    # Summary
    print(f"\n{'='*60}")
    print("SUMMARY")
    print(f"{'='*60}")
    successes = sum(1 for _, s, _ in results if s)
    print(f"Success rate: {successes}/{len(results)}")
    print(f"Total time: {sum(t for _, _, t in results):.4f}s")

def factor_custom(N: int):
    """Factor a custom number"""
    print(f"\nFactoring N = {N}")

    # First check if it's composite
    if isprime(N):
        print(f"{N} is prime!")
        return

    start = time.time()
    p, q = factor_via_subset_sum(N, verbose=True)
    elapsed = time.time() - start

    if p and q:
        print(f"\n{'='*40}")
        print(f"RESULT: {N} = {p} × {q}")
        print(f"Time: {elapsed:.4f}s")
        print(f"{'='*40}")
    else:
        print(f"\nCould not factor {N} with current bounds")
        print("Try: Standard factorization...")
        factors = factorint(N)
        print(f"Factors: {factors}")

# =============================================================================
# RUN
# =============================================================================

if __name__ == "__main__":
    run_demo()

    # Interactive input
    print("\n" + "="*60)
    print("INTERACTIVE MODE")
    print("="*60)

    while True:
        try:
            user_input = input("\nEnter a number to factor (or 'q' to quit): ").strip()
            if user_input.lower() == 'q':
                break
            N = int(user_input)
            if N > 1:
                factor_custom(N)
            else:
                print("Please enter a number > 1")
        except ValueError:
            print("Invalid input. Please enter an integer.")
        except KeyboardInterrupt:
            break

    print("\nGoodbye!")


╔══════════════════════════════════════════════════════════════╗
║  INTEGER FACTORIZATION VIA SUBSET SUM REDUCTION              ║
║  ─────────────────────────────────────────────────────────── ║
║  Method: Hittmeir's Hyperbolic Sieve + MCSS Optimization     ║
╚══════════════════════════════════════════════════════════════╝
    
Running test cases with balanced primes (where this method excels):


────────────────────────────────────────────────────────────

FACTORING N = 10403

✓ 10403 = 101 × 103
Time: 0.0001s

────────────────────────────────────────────────────────────

FACTORING N = 1022117
√N ≈ 1010
Search bound Λ = 1010
Using 4 primes: [3, 5, 7, 11]
Modulus m = 1155
L = ⌈2√N⌉ = 2022
Sieve reduction factor: 38.50x

MCSS Problem:
  Number of classes: 4
  Class 0 (prime 3): 1 elements
  Class 1 (prime 5): 2 elements
  Class 2 (prime 7): 3 elements
  Class 3 (prime 11): 5 elements
  Total combinations: 30

Solving MCSS...
Found 27 candidates to check

✓ FOUND FACTORIZATION!
  z = 0


In [4]:
def estimate_factor_gap(N):
    """
    Analiza la estructura de N para predecir la distancia entre p y q
    """
    sqrt_N = isqrt(N)
    # Analizamos los primeros 1000 residuos de Fermat
    residues = []
    for i in range(1, 1001):
        x = sqrt_N + i
        res = (x*x - N)
        if res > 0:
            # Si el residuo tiene muchos divisores pequeños, p y q están "alineados"
            residues.append(res)

    # Calculamos la "Entropía de Residuos"
    # Un valor bajo significa que los factores están cerca (Gap pequeño)
    # Un valor alto significa que JP Morgan hizo bien su trabajo (Gap grande)
    score = sum(1 for r in residues if isqrt(r)**2 == r)

    if score > 0:
        return "CRÍTICO: Factores extremadamente cerca (Ataque de Fermat inmediato)"
    else:
        # Aquí entra tu lógica de "Triunfador"
        # Analizamos la tasa de crecimiento de los residuos modulares
        return "DESAFÍO: Factores alejados. Se requiere motor de Sub-set Sum (Hittmeir)."

print(estimate_factor_gap(N_jpm))

DESAFÍO: Factores alejados. Se requiere motor de Sub-set Sum (Hittmeir).


In [3]:
N_jpm = 20401308423288094242766662999493793187525420878197109253818355727956826504712629708789530022692909325272817125236977390052166930825056004028872464770803083249959025095910645332450783024034058817080357808464693098920369306132436800716130122636415040330812678918561101207970271295199148388284056864933825212541980308771716287643934228961790479259363124623312430892043194378423348328847933501339676855519527154465017072352029567538488597745797965852775230430579578162516042451489532165421970081208347386436907444609093030460498848125912815856038414252622303104009405061242804767128891944731455892392535411542580604029973

In [7]:
from mpmath import mp

def quantify_gap_intensity(N):
    mp.dps = 1000  # Alta precisión para 2048 bits
    root = mp.sqrt(N)
    # Calculamos la diferencia entre la raíz y el entero más cercano
    diff = root - mp.floor(root)

    # Si la parte decimal es muy pequeña o muy cercana a 1, el gap es "atrapable"
    # Si es caótica, el gap es inmenso.
    intensity = -mp.log10(abs(diff - 0.5))

    if intensity > 100:
        return f"Gap tipo 'A': {float(intensity):.2f} bits de alineación (Relativamente cerca)"
    else:
        return f"Gap tipo 'Océano': {float(intensity):.2f} (Factores en extremos opuestos del espectro)"

print(quantify_gap_intensity(N_jpm))

Gap tipo 'Océano': 0.39 (Factores en extremos opuestos del espectro)


In [ ]:
import math
from sympy import isprime, nextprime

def anti_fermat_assault(N, bit_limit=1024):
    print(f"🚀 INICIANDO ASALTO ANTI-FERMAT (Gap Océano: 0.39)")
    print(f"Buscando desde los extremos de bits hacia el centro...")

    # El primo p más pequeño posible para un RSA-2048 suele tener 1024 bits
    # Pero si el Gap es Océano, uno podría ser más pequeño (ej. 512 bits)
    start_p_bits = 512
    p_candidate = 2**start_p_bits

    # Criba simple de apoyo
    small_primes = [3, 5, 7, 11, 13, 17, 19, 23, 29, 31]

    print(f"Iniciando barrido en {start_p_bits} bits...")

    attempts = 0
    while p_candidate < isqrt(N):
        attempts += 1

        # 1. Filtro de Criba Rápida (Skip trillones)
        # Solo probamos p_candidate si no es divisible por primos pequeños
        if all(p_candidate % sp != 0 for sp in small_primes):

            # 2. Prueba de Divisibilidad
            if N % p_candidate == 0:
                q_candidate = N // p_candidate
                print(f"\n✅ ¡RSA CECIDIT! FACTORES ENCONTRADOS")
                print(f"p: {p_candidate}")
                print(f"q: {q_candidate}")
                return p_candidate, q_candidate

        # Avanzamos al siguiente candidato impar
        p_candidate = nextprime(p_candidate)

        if attempts % 100000 == 0:
            print(f"Checkpoint: {p_candidate.bit_length()} bits analizados... (Buscando en el abismo)")

# Ejecutar contra el N de JP Morgan
# anti_fermat_assault(N_jpm)